In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!ls -lh /content/



total 8.0K
drwx------ 5 root root 4.0K Aug 30 21:41 drive
drwxr-xr-x 1 root root 4.0K Aug 28 13:43 sample_data


In [3]:
import time

start = time.time()
# Copy the dataset from Google Drive to the Colab working directory
!cp /content/drive/MyDrive/Dataset.zip /content/

end = time.time()
print(f"Copy completed in {round(end - start, 2)} seconds")


Copy completed in 74.21 seconds


In [4]:
import zipfile

with zipfile.ZipFile("Dataset.zip", 'r') as zip_ref:
    zip_ref.extractall("Dataset")


In [5]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import preprocess_input
from PIL import Image

# === Parameters ===
IMAGE_RESIZE = 224
model_path = '/content/drive/MyDrive/resnet50_model_1.h5'  # Path to the trained model
test_csv = '/content/Dataset/Dataset/all_test/test_category.csv'
test_dir = '/content/Dataset/Dataset/all_test/test'


In [6]:
# Load test dataset
test_df = pd.read_csv(test_csv)
test_df['label'] = test_df['label'].astype(str)  # Ensure correct class_mode format

# Load the trained model
model = load_model(model_path)

# Image preprocessing
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


In [7]:
def show_misclassified_images(cat_df, y_true, y_pred, probs, category_name):
    # Get indices of misclassified samples
    misclassified_indices = np.where(y_true != y_pred)[0]
    if len(misclassified_indices) == 0:
        print(f"No misclassified samples in {category_name}")
        return

    num_images = len(misclassified_indices)
    cols = 5
    rows = (num_images + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 3, rows * 3))
    axes = axes.flatten()
    fig.suptitle(f"Misclassified in {category_name} ({num_images} samples)", fontsize=16)

    for i, idx in enumerate(misclassified_indices):
        file_path = os.path.join(test_dir, cat_df.loc[idx, 'filename_only'])
        image = Image.open(file_path).resize((IMAGE_RESIZE, IMAGE_RESIZE))
        axes[i].imshow(image)
        axes[i].axis('off')
        # Add prediction, ground truth and probability with font size
        axes[i].set_title(f"Pred: {y_pred[idx]}, True: {y_true[idx]}\nProb: {probs[idx][0]:.2f}",
                          fontsize=8)

    for j in range(i + 1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()
    plt.show()


In [8]:
def plot_confusion_matrix(y_true, y_pred, title_name):
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
                xticklabels=['Human', 'AI'], yticklabels=['Human', 'AI'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix: {title_name}')
    plt.tight_layout()
    plt.show()


# Evaluation function
def evaluate_split(df, category_name):
    # Flow generator only accepts string labels; convert here without affecting numeric evaluation
    df_used = df.reset_index(drop=True).copy()
    df_used['label'] = df_used['label'].astype(str)

    gen = test_datagen.flow_from_dataframe(
        dataframe=df_used,
        directory=test_dir,
        x_col='filename_only',
        y_col='label',
        target_size=(IMAGE_RESIZE, IMAGE_RESIZE),
        batch_size=32,
        class_mode='binary',
        shuffle=False
    )

    pred_probs  = model.predict(gen, verbose=0)
    pred_labels = (pred_probs >= 0.5).astype(int).ravel()
    true_labels = gen.classes  # Already integer 0/1

    print(f"\nEvaluating: {category_name}")
    # Use numeric labels 0/1 directly without target_names
    from sklearn.metrics import classification_report
    print(classification_report(true_labels, pred_labels, digits=4, zero_division=0))

    plot_confusion_matrix(true_labels, pred_labels, category_name)
    show_misclassified_images(df_used, true_labels, pred_labels, pred_probs, category_name)


# Evaluate overall performance
print("\nEvaluating: Overall")
evaluate_split(test_df, "Overall")

# Evaluate performance by category
for cat in test_df['category'].dropna().unique():
    evaluate_split(test_df[test_df['category'] == cat], str(cat))


Output hidden; open in https://colab.research.google.com to view.